<a href="https://colab.research.google.com/github/yiiyama/qc-workbook-lecturenotes/blob/branch-2024/2024_04_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstration of violation of CHSH inequality

$\newcommand{\ket}[1]{|#1\rangle}$

## First, execute these cells to prepare the program

In [ ]:
# まずは必要になるpythonモジュールのライブラリファイルをコピーし、すべてインポートしておく
import os
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

runtime_config_path = '/content/gdrive/MyDrive/qiskit-ibm.json'

## Build quantum circuits to calculate CHSH inequality

In [ ]:
circuits = []

# Circuit I - H, CX[0, 1], Ry(-π/4)[1]
circuit = QuantumCircuit(2, name='circuit_I')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-np.pi / 4., 1)
circuit.measure_all()
# Added in the circuit list
circuits.append(circuit)

# Circuit II - H, CX[0, 1], Ry(-3π/4)[1]
circuit = QuantumCircuit(2, name='circuit_II')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-3. * np.pi / 4., 1)
circuit.measure_all()
# Added in the circuit list
circuits.append(circuit)

# Circuit III - H, CX[0, 1], Ry(-π/4)[1], Ry(-π/2)[0]
circuit = QuantumCircuit(2, name='circuit_III')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-np.pi / 4., 1)
circuit.ry(-np.pi / 2., 0)
circuit.measure_all()
# Added in the circuit list
circuits.append(circuit)

# Circuit IV - H, CX[0, 1], Ry(-3π/4)[1], Ry(-π/2)[0]
circuit = QuantumCircuit(2, name='circuit_IV')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-3. * np.pi / 4., 1)
circuit.ry(-np.pi / 2., 0)
circuit.measure_all()
# Added in the circuit list
circuits.append(circuit)

# Draw the circuits
fig, axs = plt.subplots(2, 2, figsize=[12., 6.])
for circuit, ax in zip(circuits, axs.reshape(-1)):
    circuit.draw('mpl', ax=ax)
    ax.set_title(circuit.name)

## Execute the circuits on quantum computer

In [ ]:
service = QiskitRuntimeService(filename=runtime_config_path)
backend = service.least_busy(filters=lambda b: not b.backend_name == 'ibm_kyoto',
                             simulator=False, operational=True)
print(f'Jobs will run on {backend.name}')

sampler = Sampler(backend)

In [ ]:
# バックエンドごとに決められている最大ショット数
shots = 10000
max_shots = backend.configuration().max_shots
print(f'Running four circuits, {shots} shots each where {max_shots} shots are allowed')

# transpileすることで、量子回路をハードウェアに実装可能なゲートに翻訳する
circuits = transpile(circuits, backend=backend, optimization_level=3)
# バックエンドで回路をshots回実行させ、測定結果を返させる
job = sampler.run(circuits, shots=shots)

### Check job status

In [ ]:
# ジョブIDとステータスを表示
print(f"Job ID: {job.job_id()}, Status: {job.status()}")

### IBM Quantumのよく使うページ

<a href="https://quantum.ibm.com/services/resources" target="_blank">バックエンド一覧</a>

- バックエンドを一つクリックすると詳細が表示される
- 現在の全ジョブ数が Total pending jobs として表示される
- Instance access limits という欄でバックエンドのジョブあたりの最大ショット数と最大回路数を確認できる

<a href="https://quantum.ibm.com/jobs" target="_blank">ジョブ一覧</a>

## Analyze the results

### Downlaod the job output

In [ ]:
result = job.result()

# 4つの回路のヒストグラムデータを入れるリスト
counts_list = []

# 回路ごとの結果をresultから抽出する
for idx in range(4):
    # get_counts(i)で回路iのヒストグラムデータが得られる
    counts = result[idx].data.meas.get_counts()
    # データをリストに足す
    counts_list.append(counts)

print(counts_list)

### Show the histograms

In [ ]:
fig, axs = plt.subplots(2, 2, sharey=True, figsize=[12., 8.])
for counts, circuit, ax in zip(counts_list, circuits, axs.reshape(-1)):
    plot_histogram(counts, ax=ax)
    ax.set_title(circuit.name)
    ax.yaxis.grid(True)

### Value of $S$

In [ ]:
# C^I, C^II, C^III, C^IVを一つのアレイにする
#（今の場合ただのリストにしてもいいが、純粋な数字の羅列にはnumpy arrayを使うといいことが多い）
c_arr = np.zeros(4, dtype=float)

# enumerate(L)でリストのインデックスと対応する要素に関するループを回せる
for ic, counts in enumerate(counts_list):
    # counts['00'] でなく counts.get('00', 0) - 上のテキストを参照
    c_arr[ic] = counts.get('00', 0) + counts.get('11', 0) - counts.get('01', 0) - counts.get('10', 0)

# 4つの要素を同時に和で規格化（リストではこういうことはできない）
c_arr /= shots

s_val = c_arr[0] - c_arr[1] + c_arr[2] + c_arr[3]

print('C:', c_arr)
print('S =', s_val)
if s_val > 2.:
    print('Yes, we are using a quantum computer!')
else:
    print('Armonk, we have a problem.')